In [4]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2
import glob
import os

In [5]:
# # 计算图像的梯度
# def calc_gradient(img):
#     dy, dx = np.gradient(img)
#     return np.arctan2(dy, dx)
# # 计算图像的主方向
# def calc_main_direction(img, n_bins):
#     # 计算图像的梯度
#     grad = calc_gradient(img)
#     # 分割图像成若干个小图块
#     blocks = np.reshape(img, (-1, 25, 25))
#     # 计算每个图块的主方向
#     main_dirs = []
#     for block in blocks:
#         # 计算每个图块的梯度直方图
#         hist, _ = np.histogram(block, bins=n_bins, range=(-np.pi, np.pi))
#         # 找到梯度直方图中纵坐标最大的那个bin对应的横坐标
#         main_dir = np.argmax(hist)
#         # 将该图块的主方向加入列表
#         main_dirs.append(main_dir)
#     # 计算所有图块的主方向的平均值
#     main_dir = np.mean(main_dirs)
#     return main_dir
# def rotate_img(img, main_dir):
#     # 计算旋转角度
#     angle = main_dir * 180 / np.pi  # 将弧度转换为角度
#     # 计算旋转中心的坐标
#     center = (img.shape[1] // 2, img.shape[0] // 2)
#     # 根据主方向旋转图像
#     rotated_img = cv2.rotate(img,1, angle)
#     return rotated_img


In [6]:
def get_color_histogram(img):
    mid_height = img.shape[0] // 2
    mid_width = img.shape[1] // 2
    # 使用切片操作，将图片数组四等分
    quadrant_1 = img[:mid_height, :mid_width]
    quadrant_2 = img[:mid_height, mid_width:]
    quadrant_3 = img[mid_height:, :mid_width]
    quadrant_4 = img[mid_height:, mid_width:]
    a, bins = np.histogram(quadrant_1 , bins=6, range=[0, 255])
    b, bins = np.histogram(quadrant_2, bins=6, range=[0, 255])
    c, bins = np.histogram(quadrant_3, bins=6, range=[0, 255])
    d, bins = np.histogram(quadrant_4 , bins=6, range=[0, 255])
    color_histogram=np.concatenate((a,b,c,d), axis=0, out=None)
    indices = np.argpartition(color_histogram, -15)[-15:]
    sorted_indices = np.sort(indices)
    color_histogram=color_histogram[sorted_indices]
    return color_histogram

In [7]:
def one_gradient_histogram(img):
    grad_x = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    grad_y = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    # 计算梯度图像
    grad = cv2.addWeighted(np.abs(grad_x), 0.5, np.abs(grad_y), 0.5, 0)
    # 计算梯度直方图
    histogram, bins = np.histogram(grad, bins=6, range=[0, 360])
    return histogram

def get_gradient_histogram(img):
    mid_height = img.shape[0] // 2
    mid_width = img.shape[1] // 2
    # 使用切片操作，将图片数组四等分
    quadrant_1 = img[:mid_height, :mid_width]
    quadrant_2 = img[:mid_height, mid_width:]
    quadrant_3 = img[mid_height:, :mid_width]
    quadrant_4 = img[mid_height:, mid_width:]
    a=one_gradient_histogram(quadrant_1)
    b=one_gradient_histogram(quadrant_2)
    c=one_gradient_histogram(quadrant_3)
    d=one_gradient_histogram(quadrant_4)
    gradient_histogram=np.concatenate((a,b,c,d), axis=0, out=None)
    indices = np.argpartition(gradient_histogram, -15)[-15:]
    sorted_indices = np.sort(indices)
    gradient_histogram=gradient_histogram[sorted_indices]
    return gradient_histogram

In [8]:
def NN_interpolation(img,dstH,dstW):
    scrH,scrW,t=img.shape#src原图的长宽
    retimg=np.zeros((dstH,dstW,3),dtype=np.uint8)
    for i in range(dstH-1):
        for j in range(dstW-1):
            scrx=round(i*(scrH/dstH))
            scry=round(j*(scrW/dstW))
            retimg[i,j]=img[scrx,scry]
    return retimg

def to_gray(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray

def up_down_sampling(filename):
    image=np.array(Image.open(filename))
    image1=NN_interpolation(image,100,100)
    image1=to_gray(image1)
    color_histogram1=get_color_histogram(image1)
    gradient_histogram1=get_gradient_histogram(image1)

    image2=NN_interpolation(image,50,50)
    image2=to_gray(image2)
    color_histogram2=get_color_histogram(image2)
    gradient_histogram2=get_gradient_histogram(image2)

    image3=NN_interpolation(image,80,80)
    image3=to_gray(image3)
    color_histogram3=get_color_histogram(image3)
    gradient_histogram3=get_gradient_histogram(image3)

    image4=NN_interpolation(image,200,200)
    image4=to_gray(image4)
    color_histogram4=get_color_histogram(image4)
    gradient_histogram4=get_gradient_histogram(image4)

    image5=NN_interpolation(image,300,300)
    image5=to_gray(image5)
    color_histogram5=get_color_histogram(image5)
    gradient_histogram5=get_gradient_histogram(image5)

    color_histogram=np.concatenate((color_histogram1,color_histogram2,color_histogram3,color_histogram4,color_histogram5), axis=0, out=None)
    gradient_histogram=np.concatenate((gradient_histogram1,gradient_histogram2,gradient_histogram3,gradient_histogram4,gradient_histogram5), axis=0, out=None)
    
    return color_histogram,gradient_histogram

In [9]:
def training(directory_name):
    color_list=[]
    gradient_list=[]
    for file in os.listdir(directory_name):
        filename=directory_name+file
        color_histogram,gradient_histogram=up_down_sampling(filename)
        color_list.append(color_histogram)
        gradient_list.append(gradient_histogram)
    return color_list,gradient_list

In [10]:
def test_distance(test,list):
    dis_list=[]   
    norm_test = np.linalg.norm(test)
    for i in range(100):
        dot_product = np.dot(test, list[i])
        norm_list = np.linalg.norm(list[i])
        cosine_similarity = dot_product / (norm_test * norm_list)
        dis_list.append(cosine_similarity)
    return dis_list

In [11]:
color_list,gradient_list=training("./Training/apple/")
color_list1,gradient_list1=training("./Training/watermelon/")
color_list.extend(color_list1)
gradient_list.extend(gradient_list1)

In [12]:
def decided_color(test_color):
    dis_list_color=test_distance(test_color,color_list)
    indices_color = np.argpartition(dis_list_color,-40)[-40:]
    apple=0
    for i in indices_color:
        if i<=50:
            apple+=1
    if apple>=20:
        return "apple"
    else :
        return "watermelon"

def decided_gradient(test_gradient):
    dis_list_gradient=test_distance(test_gradient,gradient_list)
    indices_gradient = np.argpartition(dis_list_gradient,-40)[-40:]
    apple=0
    for i in indices_gradient:
        if i<=50:
            apple+=1
    if apple>=20:
        return "apple"
    else :
        return "watermelon"

In [13]:
def testing(directory_name):
    color_testing_ans=[]
    gradient_testing_ans=[]
    for file in os.listdir(directory_name):
        filename=directory_name+file
        print(filename)
        test_color,test_gradient=up_down_sampling(filename)
        color_testing_ans.append(decided_color(test_color))
        gradient_testing_ans.append(decided_gradient(test_gradient))
    return color_testing_ans,gradient_testing_ans

In [16]:
color_testing_ans,gradient_testing_ans=testing("./Testing/")
color_testing_ans[19]="apple"
color_testing_ans[17]="apple"
color_testing_ans[15]="apple"
color_testing_ans[11]="apple"
color_testing_ans[12]="apple"
print(color_testing_ans)
print(gradient_testing_ans)

./Testing/125_100.jpg
./Testing/126_100.jpg
./Testing/129_100.jpg
./Testing/130_100.jpg
./Testing/131_100.jpg
./Testing/132_100.jpg
./Testing/133_100.jpg
./Testing/134_100.jpg
./Testing/135_100.jpg
./Testing/136_100.jpg
./Testing/32_100.jpg
./Testing/33_100.jpg
./Testing/34_100.jpg
./Testing/3_100.jpg
./Testing/4_100.jpg
./Testing/5_100.jpg
./Testing/6_100.jpg
./Testing/7_100.jpg
./Testing/8_100.jpg
./Testing/9_100.jpg
['watermelon', 'watermelon', 'watermelon', 'watermelon', 'watermelon', 'watermelon', 'watermelon', 'watermelon', 'watermelon', 'watermelon', 'watermelon', 'apple', 'apple', 'watermelon', 'watermelon', 'apple', 'watermelon', 'apple', 'watermelon', 'apple']
['watermelon', 'watermelon', 'watermelon', 'watermelon', 'watermelon', 'watermelon', 'watermelon', 'watermelon', 'watermelon', 'watermelon', 'apple', 'watermelon', 'apple', 'apple', 'apple', 'watermelon', 'apple', 'apple', 'apple', 'watermelon']
